# Music-Source-Separation-Training-Inference-Webui For Google Colab

LICENSE: AGPL-3.0

For personal entertainment and non-commercial use only. Prohibited from use in content related to blood, gore, violence, sex, politics.<br>
Author: [Github@KitsuneX07](https://github.com/KitsuneX07) | [Github@SUC-DriverOld](https://github.com/SUC-DriverOld) | Gradio theme: [Gradio Theme](https://huggingface.co/spaces/NoCrypt/miku)<br>
Github: [https://github.com/SUC-DriverOld/MSST-WebUI](https://github.com/SUC-DriverOld/MSST-WebUI)

## Introduction

This is a webUI for [Music-Source-Separation-Training (MSST)](https://github.com/ZFTurbo/Music-Source-Separation-Training), which is a repository for training models for music source separation. You can use this webUI to infer the MSST model and VR Models, and the preset process page allows you to customize the processing flow yourself. You can install models in the "Install Models" interface. If you have downloaded [Ultimate Vocal Remover (UVR)](https://github.com/Anjok07/ultimatevocalremovergui) before, you do not need to download VR Models again. You can go to the "Settings" page and directly select your UVR5 model folder. We also provide some convenient tools in the WebUI such as [Singing-Oriented MIDI Extractor (SOME)](https://github.com/openvpi/SOME/), advanced ensemble mode, and more.

In [1]:
# @title Connect to colab runtime and check GPU
# @markdown # Connect to colab runtime and check GPU
# @markdown

!nvidia-smi

Tue Jan 20 08:43:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# @title Install python 3.10, ffmpeg
# @markdown # Install python 3.10, ffmpeg
# @markdown

!sudo apt install python3.10
!sudo apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10 is already the newest version (3.10.12-1~22.04.12).
python3.10 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
# @title Clone repository and install requirements
# @markdown # Clone repository and install requirements
# @markdown
# @markdown ### You may need to rerun this cell for a few times
# @markdown

# Clone the repository
%cd /content
!git clone https://github.com/SUC-DriverOld/MSST-WebUI
%cd /content/MSST-WebUI
!git checkout ccbce58

# Upgrade pip and setuptools
%pip install --upgrade pip setuptools

# Install requirements
%pip install tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
%pip install -r requirements.txt --only-binary=samplerate

/content
Cloning into 'MSST-WebUI'...
remote: Enumerating objects: 2826, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 2826 (delta 177), reused 146 (delta 145), pack-reused 2568 (from 3)
Receiving objects: 100% (2826/2826), 4.75 MiB | 22.51 MiB/s, done.
Resolving deltas: 100% (1785/1785), done.
/content/MSST-WebUI
Note: switching to 'ccbce58'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ccbce58 Fix error unable find mps if use --cloud 

Processing ./tools/webUI_for_clouds/librosa-0.9.2-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s  0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.11.0
    Uninstalling librosa-0.11.0:
      Successfully uninstalled librosa-0.11.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [librosa]
^C


In [ ]:
# @title [Optional] Download SOME weights if needed
# @markdown # Download SOME weights if needed
# @markdown
# @markdown ### If you want to use "Vocals to MIDI", download the weights.
# @markdown
# @markdown ### "Vocals to MIDI" use project: [SOME](https://github.com/openvpi/SOME/).
# @markdown

!wget https://huggingface.co/Sucial/MSST-WebUI/resolve/main/SOME_weights/model_steps_64000_simplified.ckpt -O /content/MSST-WebUI/tools/SOME_weights/model_steps_64000_simplified.ckpt

In [ ]:
# @title [Optional] Pack and upload your music files to your google drive
# @markdown # Pack and upload your music files to your google drive
# @markdown
# @markdown Ensure that there are no nested folders in your zip file, but rather direct audio files.
# @markdown

from google.colab import drive

drive.mount("/content/drive")
# @markdown Directory where **your zip file** located in, dont miss the slash at the end.
data_dir = "/content/drive/MyDrive/MSST-WebUI/"  # @param {type:"string"}
# @markdown Filename of **your zip file**, for example: "inputs.zip"
zip_filename = "input.zip"  # @param {type:"string"}
ZIP_PATH = data_dir + zip_filename
# @markdown You need to package the audio directly into a zip file, **DO NOT** nest folders in the zip file!
# @markdown The compressed file will be extracted to the `input` directory. If there are files in your `input` directory, they will be deleted. Please backup in advance.
# @markdown

!rm -rf /content/MSST-WebUI/input
!mkdir -p /content/MSST-WebUI/input
!unzip -od /content/MSST-WebUI/input {ZIP_PATH}

In [ ]:
# @title [Optional] Save results to google drive
# @markdown # Save results to google drive
# @markdown
# @markdown The results audio files in the output directory will be compressed into a zip file and saved to your google drive.
# @markdown

from google.colab import drive

drive.mount("/content/drive")
# @markdown Path to save the zip file, dont miss the slash at the end.
output_dir = "/content/drive/MyDrive/MSST-WebUI/"  # @param {type:"string"}
# @markdown Filename of the zip file, for example: "Outputs.zip"
zip_filename = "Outputs.zip"  # @param {type:"string"}
ZIP_PATH = output_dir + zip_filename

!mkdir -p {output_dir}
!zip -r "{zip_filename}" /content/MSST-WebUI/results
!cp -vr "{zip_filename}" {output_dir}

In [ ]:
# @title [Optional] Delete all result audio files
# @markdown # Click to delete all result audio files
# @markdown
# @markdown ### Warning: Unrecoverable, please operate with caution
# @markdown

!rm -rf /content/MSST-WebUI/results
!mkdir -p /content/MSST-WebUI/results

In [ ]:
# @title [Optional] Delete all input audio files
# @markdown # Click to delete all input audio files
# @markdown
# @markdown ### Warning: Unrecoverable, please operate with caution
# @markdown

!rm -rf /content/MSST-WebUI/input
!mkdir -p /content/MSST-WebUI/input

In [ ]:
# @title Initialize and start WebUI
# @markdown # Initialize and start WebUI
# @markdown
# @markdown After running this cell, you can use your own device to connect to WebUI. The public link will be displayed below.
# @markdown
# @markdown If you want to modify the inference parameters of the preset process, edit the `data/webui_config.json` file.
# @markdown

%cd /content/MSST-WebUI

LANGUAGE = "English"  # @param ["Auto", "English", "简体中文", "繁體中文", "日本語", "😊", "한국어"]
MODEL_DOWNLOAD_LINK = "huggingface.co"  # @param ["Auto", "huggingface.co", "hf-mirror.com"]
DEBUG = False  # @param {type:"boolean"}

language_dict = {"Auto": "Auto", "简体中文": "zh_CN", "繁體中文": "zh_TW", "English": "en_US", "日本語": "ja_JP", "😊": "emoji", "한국어": "ko_KR"}
language = language_dict[LANGUAGE]
debug = "--debug" if DEBUG else ""

# Start WebUI
!python webUI.py --use_cloud --share --language {language} --model_download_link {MODEL_DOWNLOAD_LINK} {debug}